**Table of contents**<a id='toc0_'></a>    
- [Libraries](#toc1_)    
- [Loading text filings](#toc2_)    
- [Extracting the Management's Discussion and Analysis Section](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [11]:
import os
import time
os.chdir(os.environ.get('PROJECT_PATH'))
from secnlp.ml_logic import data as d
from secnlp.ml_logic import parsing as p
import secnlp.ml_logic.parsing
from secnlp import utils as u
from secnlp.params import *
import pandas as pd
import importlib

# <a id='toc2_'></a>[Loading text filings](#toc0_)

In [2]:
df = u.read_data_from_bq(credentials = SERVICE_ACCOUNT, gcp_project = PROJECT, bq_dataset = DATASET_ID, table = FILINGS_10KQ_TABLE_ID)

In [3]:
df['date_filed'] = pd.to_datetime(df['date_filed'])

In [41]:
filing_sample_10k = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-K')].sample(1)
filing_sample_10k['raw_filing'] = filing_sample_10k['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


In [80]:
filing_sample_10q = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-Q')].sample(1)
filing_sample_10q['raw_filing'] = filing_sample_10q['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


# 10-Ks

In [86]:
text_10k = filing_sample_10k['raw_filing'].iloc[0]

In [87]:
mda_10k = p.parse_10k_filing_items(text_10k, item = '7')

Write a function that will delete all of the original markup language tags (HTML, XBRL, XML):
- [ ] Remove ASCII-Encoded segments – All document segment <TYPE> tags of GRAPHIC, ZIP, EXCEL, JSON, and PDF are deleted from the file. ASCII-encoding is a means of converting binary-type files into standard ASCII characters to facilitate transfer across various hardware platforms. A relatively small graphic can create a substantial ASCII segment. Filings containing multiple graphics can be orders of magnitude larger than those containing only textual information.
- [ ] Remove <DIV>, <TR>, <TD>, and <FONT> tags – Although we require some HTML information for subsequent parsing, the files are so large (and processed as a single string) that, for processing efficiency, we initially simply strip out some of the formatting HTML.
- [ ] Remove all XML – all XML embedded documents are removed.
- [ ] Remove all XBRL – all characters between <XBRL …> … </XBRL> are deleted.
- [ ] Remove SEC Header/Footer – All characters from the beginning of the original file thru </SEC-HEADER> (or </IMS-HEADER> in some older documents) are deleted from the file. Note however that the header information is retained and included in the tagged items discussed in section 4.1. In addition, the footer “-----END PRIVACY-ENHANCED MESSAGE-----” appearing at the end of each document is deleted.
- [ ] Replace \&NBSP and \&#160 with a blank space.
- [ ] Replace \&AMP and \&#38 with “&”
- [ ] Remove all remaining extended character references (ISO-8859-1, see https://www.sec.gov/info/edgar/specifications/edgarfm-vol2-v59.pdf, section "5.2.2.6 Extended Character Sets within HTML Documents".
Tag Exhibits – At this point in the parsing process all exhibits are tagged as discussed in section 3.2.
- [ ] Remove Markup Tags – remove all remaining markup tags (i.e., <…>).
- [ ] Remove excess linefeeds.

# 10-Q

In [84]:
text_10q = filing_sample_10q['raw_filing'].iloc[0]
mda_10q = p.parse_10q_filing_items(text_10q, item = '2')

In [102]:
p.cleaning(mda_10k)

'item 7 management8217s discussion and analysis of financial condition and results of operations this form 10k contains forwardlooking statements within the meaning of section 27a of the securities act of 1933 and section 21e of the securities exchange act of 1934 including statements that involve expectations plans or intentions such as those relating to future business future results of operations or financial condition new or planned features or services mergers or acquisitions or management strategies these forwardlooking statements can be identified by words such as 8220may8221 8220will8221 8220would8221 8220should8221 8220could8221 8220expect8221 8220anticipate8221 8220believe8221 8220estimate8221 8220intend8221 continue 8220strategy8221 8220future8221 8220opportunity8221 8220plan8221 8220project8221 8220forecast8221 and other similar expressions these forwardlooking statements involve risks and uncertainties that could cause our actual results and financial condition to differ m

In [101]:
importlib.reload(secnlp.ml_logic.parsing)


<module 'secnlp.ml_logic.parsing' from '/Users/Debora/code/dramella/project/secnlp/ml_logic/parsing.py'>